<a href="https://colab.research.google.com/github/matdcp/Data-dreamers-/blob/main/NEURAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Upload cleaned dataset


In [1]:
import pandas as pd
from google.colab import files

# Carica il file CSV
uploaded = files.upload()  # Ti verrà chiesto di selezionare il file dal tuo computer



Saving cleaned_dataset_NN.csv to cleaned_dataset_NN (2).csv


In [2]:
# Leggi il file CSV con Pandas
for filename in uploaded.keys():
    df = pd.read_csv(filename)

df.head()

,id,Start,End,Label
0,Ventricular,0,11,B-Multi-tissue_structure
1,fibrillation,12,24,O
2,long,32,36,O
3,QT,37,39,O
4,syndrome,40,48,O


#Data preprocessing

In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

# **1. Definizione dei gruppi di token e delle etichette**
sequence_length = 10  # Definisci il numero di token per frase
df['Group'] = df.index // sequence_length  # Crea un gruppo ogni 10 token

# Gruppo delle sequenze e delle etichette
sentences = df.groupby('Group')['id'].apply(list).tolist()  # Liste di token per gruppo
labels = df.groupby('Group')['Label'].apply(list).tolist()  # Liste di etichette per gruppo

# **2. Tokenizzazione delle sequenze**
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)  # Addestra il tokenizer sulle sequenze di token

# Converti i token in ID numerici
X = tokenizer.texts_to_sequences(sentences)

# **3. Codifica delle etichette con LabelEncoder**
# Addestra il LabelEncoder su tutte le etichette del dataset
label_encoder = LabelEncoder()
label_encoder.fit(df['Label'])  # Usa tutte le etichette presenti nella colonna 'Label'

# Converte le etichette in ID numerici per ogni frase
y = [label_encoder.transform(sentence) for sentence in labels]

# **4. Applicazione del padding**
# Lunghezza massima delle sequenze (per token e etichette)
max_sequence_len = 10  # Usa 10 come lunghezza massima (già definita)

# Applica il padding alle sequenze di token
X_padded = pad_sequences(X, maxlen=max_sequence_len, padding='post')

# Applica il padding alle sequenze di etichette
y_padded = pad_sequences(y, maxlen=max_sequence_len, padding='post')

# **5. Verifiche**
print(f"Shape di X_padded: {X_padded.shape}")
print(f"Shape di y_padded: {y_padded.shape}")

# Esempio di sequenze
print("\nEsempio di sequenza tokenizzata e con padding (X_padded):")
print(X_padded[:3])  # Mostra le prime 3 sequenze

# Esempio di etichette
print("\nEsempio di etichette con padding (y_padded):")
print(y_padded[:3])  # Mostra le prime 3 etichette

# Visualizza i token originali per la prima sequenza
print("\nToken originali per la prima sequenza (X_padded):")
print([tokenizer.index_word[idx] for idx in X_padded[0] if idx != 0])  # Ignora i padding (0)

# Visualizza le etichette originali per la prima sequenza
print("\nEtichette originali per la prima sequenza (y_padded):")
original_labels = [
    label_encoder.inverse_transform([idx])[0] for idx in y_padded[0] if idx != 0
]
print(original_labels)


Shape di X_padded: (6098, 10)
Shape di y_padded: (6098, 10)

Esempio di sequenza tokenizzata e con padding (X_padded):
[[1187 2898  309 1740  259 2026  463 2899 2900 1740]
 [  36  397 6456  432   58  886  819 6457  887  888]
 [ 173 1740   36  397 2900   49 2899  525  889 1187]]

Esempio di etichette con padding (y_padded):
[[ 5 22 22 22 22 22 22 22 22 22]
 [22 22 22 22 22 22 22 22 22 22]
 [22 22 22 22 22 22 22 22 22  5]]

Token originali per la prima sequenza (X_padded):
['ventricular', 'fibrillation', 'long', 'qt', 'syndrome', 'probably', 'caused', 'clindamycin', 'prolongation', 'qt']

Etichette originali per la prima sequenza (y_padded):
['B-Multi-tissue_structure', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


#Splitting dataset

In [12]:
from sklearn.model_selection import train_test_split

# Esegui lo split su X_padded e y_padded
X_train, X_temp, y_train, y_temp = train_test_split(X_padded, y_padded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train set: X={X_train.shape}, y={y_train.shape}")
print(f"Validation set: X={X_val.shape}, y={y_val.shape}")
print(f"Test set: X={X_test.shape}, y={y_test.shape}")



Train set: X=(4268, 10), y=(4268, 10)
Validation set: X=(915, 10), y=(915, 10)
Test set: X=(915, 10), y=(915, 10)


#Rebalancing dataset

In [13]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Converti y_train in un array 1D per calcolare i pesi
y_train_flat = y_train.flatten()
unique_labels = np.unique(y_train_flat)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=unique_labels,
    y=y_train_flat
)

class_weights_dict = dict(enumerate(class_weights))
print("Pesi delle classi:", class_weights_dict)


Pesi delle classi: {0: 47.58082497212932, 1: 3.3495526604928583, 2: 13.446754883427852, 3: 88.36438923395445, 4: 40.34026465028355, 5: 4.034026465028355, 6: 7.277067348678602, 7: 14.727398205659075, 8: 8.921404682274247, 9: 7.277067348678602, 10: 15.860274990709774, 11: 92.78260869565217, 12: 5.674777290253956, 13: 36.38533674339301, 14: 463.9130434782609, 15: 142.74247491638795, 16: 6.82225063938619, 17: 43.15470171890799, 18: 142.74247491638795, 19: 35.68561872909699, 20: 11.314952279957582, 21: 24.416475972540045, 22: 0.04706671166014923}


In [14]:
from tensorflow.keras.utils import to_categorical

# Numero di classi
num_classes = len(label_encoder.classes_)

# Convertire le etichette in formato one-hot
y_train = np.array([to_categorical(seq, num_classes=num_classes) for seq in y_train])
y_val = np.array([to_categorical(seq, num_classes=num_classes) for seq in y_val])
y_test = np.array([to_categorical(seq, num_classes=num_classes) for seq in y_test])

# Verifica delle nuove shape
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"y_test shape: {y_test.shape}")


y_train shape: (4268, 10, 23)
y_val shape: (915, 10, 23)
y_test shape: (915, 10, 23)


#Neural Network Approach
##Embedding  
###GloVe

In [8]:
# Scarica il file GloVe e decomprimilo
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

# Specifica il percorso del file GloVe
glove_path = 'glove.6B.100d.txt'

# Funzione per caricare i vettori pre-addestrati GloVe
def load_glove(glove_path):
    embedding_dict = {}
    with open(glove_path, 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embedding_dict[word] = vector
    return embedding_dict

# Carica i vettori GloVe
embedding_dict = load_glove(glove_path)
print(f"Numero di parole caricate da GloVe: {len(embedding_dict)}")


# Creazione della matrice di embedding
embedding_dim = 100  # Assicurati che corrisponda alla dimensione dei vettori GloVe
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, idx in tokenizer.word_index.items():
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx] = embedding_vector

print(f"Matrice di embedding creata con shape: {embedding_matrix.shape}")


--2024-12-10 15:16:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-12-10 15:16:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-12-10 15:16:17--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

##LSTM

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Modello GloVe + LSTM
model = Sequential([
    Embedding(input_dim=embedding_matrix.shape[0],  # Dimensione del vocabolario
              output_dim=embedding_dim,            # Dimensione degli embedding
              weights=[embedding_matrix],          # Matrice di embedding pre-addestrata
              input_length=X_train.shape[1],       # Lunghezza massima delle sequenze
              trainable=False),                    # Non aggiorniamo GloVe durante il training
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    TimeDistributed(Dense(64, activation='relu')),  # TimeDistributed per applicare il Dense a ogni token
    Dropout(0.5),
    TimeDistributed(Dense(y_train.shape[2], activation='softmax'))  # Numero di classi target
])

# Compilazione del modello
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Riepilogo del modello
model.summary()

# Callback per Early Stopping e Checkpoint
early_stopping = EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath='best_model.keras',  # Cambiato da .h5 a .keras
    monitor='val_loss',
    save_best_only=True
)




Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │       1,152,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,152,000 (4.39 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,152,000 (4.39 MB)